# A new Pipeline Approach: CORRAL


## The TORITOS Pipeline's Engine

### Co-workers: J. B. Cabral, M. Beroiz, TOROS Collaboration

## What is a pipeline?

*A pipeline can be understood in a collection of filters and connectors that consume data to perform a transformation of it on a linear and repetitive fashion.*

This data that is consumed can be named as a *stream* and each filter is a transformation. Connectors between the filters are the schema of the pipeline, and it inspires a *always forward* structure.

<img src='./images/pipeline.png'>

## Why do we do this?


* We need this for TOROS/TORITOS

* Observational astronomers are facing a data *tsunami*, and synoptic surveys are one of the more data intensive tasks today.

* Telescopes of any size can produce massive amounts of data. 

* Processing is time consuming for humans, and most of the tasks involved are likely to fail.

## The TORITOS Project

* The first observation campaign needs.

    * A server to host data
    * A data record management
    * A computing system
    * Automatization of every process
    * A report system (lack of connectivity)
    * Error handling mechanisms

## CORRAL

**Corral is a framework designed to create fully integrated pipelines without tears and headaches.**

It makes use of some computational formalisms and tools:
   * MVC pattern
   * ETL operation routines
   * OOP (Python native)
   * SQL database fully integrated
   * Pipeline branching
   * Asynchronus and embarrasing parallel processing

## Toritos pipeline design

The TORITOS pipeline needs three central entities:
* The **models**

* The **loader** and **steps**

* The **alerts**